In [4]:
! pip uninstall mlflow
! pip install gorilla
! pip install pytorch-lightning
! conda install -c conda-forge/label/cf202003 mlflow --yes
! pip3 install mlflow
! pip install torchvision

Found existing installation: mlflow 1.9.1
Uninstalling mlflow-1.9.1:
  Would remove:
    /home/ec2-user/anaconda3/envs/python3/bin/mlflow
    /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/mlflow-1.9.1.dist-info/*
    /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/mlflow/*
  Would not remove (might be manually added):
    /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/mlflow/server/js/build/static/css/main.6fe85607.css
    /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/mlflow/server/js/build/static/js/main.d79a6b5a.js
Proceed (y/n)? ^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
  Using cached pytorch_lightning-0.8.5-py3-none-any.whl (313 kB)
  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
     |████████████████████████████████| 88 kB 845 kB/s eta 0:00:011
     |██████████████████████

s3transfer-0.1.13    | 79 KB     | ##################################### | 100% 
botocore-1.12.189    | 2.6 MB    | ##################################### | 100% 
docutils-0.16        | 668 KB    | ##################################### | 100% 
boto3-1.9.66         | 105 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/e

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    

In [1]:
import gorilla
import logging
import mlflow
import os
import pytorch_lightning as pl
import shutil
import tempfile
import torch
from mlflow.utils.autologging_utils import try_mlflow_log
from pytorch_lightning.callbacks import EarlyStopping
from torchvision import datasets, transforms
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

logging.basicConfig(level=logging.ERROR)

In [2]:

class LightningMNISTClassifier(pl.LightningModule):
    def __init__(self):
        """
        Initializes the network
        """
        super(LightningMNISTClassifier, self).__init__()

        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)

    def forward(self, x):
        """
        Forward Function
        """
        batch_size, channels, width, height = x.size()

        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)

        # layer 1 (b, 1*28*28) -> (b, 128)
        x = self.layer_1(x)
        x = torch.relu(x)

        # layer 2 (b, 128) -> (b, 256)
        x = self.layer_2(x)
        x = torch.relu(x)

        # layer 3 (b, 256) -> (b, 10)
        x = self.layer_3(x)

        # probability distribution over labels
        x = torch.log_softmax(x, dim=1)

        return x

    def cross_entropy_loss(self, logits, labels):
        """
        Loss Fn to compute loss
        """
        return F.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        """
        training the data as batches and returns training loss on each batch
        """
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {"loss": loss}

    def validation_step(self, val_batch, batch_idx):
        """
        Performs validation of data in batches
        """
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        """
        Computes average validation accuracy
        """
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, test_batch, batch_idx):
        """
        Performs test and computes test accuracy
        """
        x, y = test_batch
        output = self.forward(x)
        a, y_hat = torch.max(output, dim=1)
        from sklearn.metrics import accuracy_score

        test_acc = accuracy_score(y_hat.cpu(), y.cpu())
        return {"test_acc": torch.tensor(test_acc)}

    def test_epoch_end(self, outputs):
        """
        Computes average test accuracy score
        """
        avg_test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        return {"avg_test_acc": avg_test_acc}

    def prepare_data(self):
        """
        Preprocess the input data.
        """
        # transforms for images
        transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )

        mnist_train = datasets.MNIST(
            "datasets", download=True, train=True, transform=transform
        )
        self.mnist_test = datasets.MNIST(
            "datasets", download=True, train=False, transform=transform
        )

        self.mnist_train, self.mnist_val = random_split(mnist_train, [55000, 5000])

    def train_dataloader(self):
        """
        Loading training data as batches
        """
        return DataLoader(self.mnist_train, batch_size=64)

    def val_dataloader(self):
        """
        Loading validation data as batches
        """
        return DataLoader(self.mnist_val, batch_size=64)

    def test_dataloader(self):
        """
        Loading test data as batches
        """
        return DataLoader(self.mnist_test, batch_size=64)

    def configure_optimizers(self):
        """
        Creates and returns Optimizer
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [3]:

def autolog():
    """
    Enable automatic logging from Pytorch-Lightning to MLflow.
    Logs loss and any other metrics specified in the fit
    function, and optimizer data as parameters. Model checkpoints
    are logged as artifacts to a 'models' directory.

    MLflow will also log the parameters of the EarlyStopping callback
    """
    class __MLflowPLCallback(pl.Callback):
        """
        Callback for auto-logging metrics and parameters.
        """
        def __init__(self):
            super().__init__()

        def on_epoch_end(self, trainer, pl_module):
            """
            Log loss and other metrics values after each epoch
            """
            loss_metrics = trainer.callback_metrics

            for key, value in loss_metrics.items():
                if "loss" in key:
                    try_mlflow_log(mlflow.log_metric, key, float(value))

        def on_train_start(self, trainer, pl_module):
            """
            Logs Optimizer related metrics when the train begins
            """
            if hasattr(trainer, "optimizers"):
                for optimizer in trainer.optimizers:
                    try_mlflow_log(
                        mlflow.log_param, "optimizer_name", type(optimizer).__name__
                    )
                    if hasattr(optimizer, "defaults"):
                        defaults_dict = optimizer.defaults
                        if "lr" in defaults_dict:
                            try_mlflow_log(
                                mlflow.log_param, "learning_rate", defaults_dict["lr"]
                            )

                        if "eps" in defaults_dict:
                            try_mlflow_log(
                                mlflow.log_param, "epsilon", defaults_dict["eps"]
                            )

            summary = str(trainer.model.summarize("full"))
            tempdir = tempfile.mkdtemp()
            try:
                summary_file = os.path.join(tempdir, "model_summary.txt")
                with open(summary_file, "w") as f:
                    f.write(summary)

                try_mlflow_log(mlflow.log_artifact, local_path=summary_file)
            finally:
                shutil.rmtree(tempdir)

        def on_train_end(self, trainer, pl_module):
            """
            Logs the model checkpoint into mlflow - models folder
            """

            model_file_name = "model.ckpt"
            trainer.save_checkpoint(model_file_name)
            try_mlflow_log(
                mlflow.log_artifact, local_path=model_file_name, artifact_path="models"
            )

        def on_test_end(self, trainer, pl_module):
            """
            Logs accuracy and other testing metrics on the training end
            """
            loss_metrics = trainer.callback_metrics

            for key, value in loss_metrics.items():
                if key != "epoch":
                    try_mlflow_log(mlflow.log_metric, key, float(value))


    def _log_early_stop_params(early_stop_obj):
        """
        Logs Early Stop parameters into mlflow
        """
        if hasattr(early_stop_obj, "monitor"):
            try_mlflow_log(mlflow.log_param, "monitor", early_stop_obj.monitor)
        if hasattr(early_stop_obj, "mode"):
            try_mlflow_log(mlflow.log_param, "mode", early_stop_obj.mode)
        if hasattr(early_stop_obj, "patience"):
            try_mlflow_log(mlflow.log_param, "patience", float(early_stop_obj.patience))
        if hasattr(early_stop_obj, "min_delta"):
            try_mlflow_log(
                mlflow.log_param, "min_delta", float(early_stop_obj.min_delta)
            )
        if hasattr(early_stop_obj, "stopped_epoch"):
            try_mlflow_log(
                mlflow.log_param, "stopped_epoch", float(early_stop_obj.stopped_epoch)
            )

    def _run_and_log_function(
        self, original, args, kwargs
    ):
        """
        This method would be called from patched fit method and
        It adds the custom callback class to the callback list by default.
        """
        early_stop_obj = gorilla.get_attribute(self, "early_stop_callback")

        if isinstance(early_stop_obj, EarlyStopping):
            _log_early_stop_params(early_stop_obj)

        self.callbacks += [__MLflowPLCallback()]

        result = original(self, *args, **kwargs)

        return result

    @gorilla.patch(pl.Trainer)
    def fit(self, *args, **kwargs):
        """
        Patching trainer.fit method to add autolog class into callback
        """
        original = gorilla.get_original_attribute(pl.Trainer, "fit")
        return _run_and_log_function(self, original, args, kwargs)

    settings = gorilla.Settings(allow_hit=True, store_hit=True)
    gorilla.apply(gorilla.Patch(pl.Trainer, "fit", fit, settings=settings))

In [4]:
if __name__ == "__main__":
    mlflow.tracking.set_tracking_uri("http://ec2-3-21-204-186.us-east-2.compute.amazonaws.com:5000")
    mlflow.set_experiment("mnist")
    if not mlflow.active_run():
        try_mlflow_log(mlflow.start_run)

    model = LightningMNISTClassifier()
    trainer = pl.Trainer(max_epochs=5, callbacks=[])
    autolog()
    trainer.fit(model)
    trainer.test(model)

    try_mlflow_log(mlflow.end_run)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores

  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   
INFO:lightning:
  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   
INFO:lightning:
  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33 K  
2 | layer_3 | Linear | 2 K   


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_acc': tensor(0.9721, dtype=torch.float64)}
--------------------------------------------------------------------------------

